This script builds, trains, and evaluates a nueral network surrogate model to predict the deflection of a beam based on the generated dataset.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import time

In [9]:
# --- 1. Load Data ---
print("Loadind dataset...")
df = pd.read_csv(r"C:\Users\pasch\OneDrive\PortalFrameAnalysis\beam_deflection_dataset.csv")
print("Dataset loaded successfully.")
print("\nDataset Info:")
df.info()
print("\nFirst 5 rows of the dataset:")
print(df.info())

Loadind dataset...
Dataset loaded successfully.

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   k0        2000 non-null   float64
 1   k1        2000 non-null   float64
 2   damping   2000 non-null   float64
 3   velocity  2000 non-null   float64
 4   w_max     2000 non-null   float64
dtypes: float64(5)
memory usage: 78.3 KB

First 5 rows of the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   k0        2000 non-null   float64
 1   k1        2000 non-null   float64
 2   damping   2000 non-null   float64
 3   velocity  2000 non-null   float64
 4   w_max     2000 non-null   float64
dtypes: float64(5)
memory usage: 78.3 KB
None


In [6]:
df

,k0,k1,damping,velocity,w_max
0,1.073641e+08,148746.025235,0.020847,20.553105,0.000084
1,2.277191e+07,84102.166772,0.065196,25.656706,0.000268
2,9.175781e+07,31225.232905,0.171639,29.383616,0.000095
3,3.972292e+07,107823.087179,0.047984,62.910787,0.000183
4,1.901094e+07,157633.258648,0.018515,72.599706,0.000309
...,...,...,...,...,...
1995,1.340495e+08,68238.118267,0.081144,71.546504,0.000073
1996,7.674841e+07,118055.199765,0.024334,48.119528,0.000110
1997,1.322220e+07,153852.980762,0.080516,60.188139,0.000402
1998,1.885010e+08,69389.389191,0.194409,10.906861,0.000055


In [7]:
df.head()

,k0,k1,damping,velocity,w_max
0,1.073641e+08,148746.025235,0.020847,20.553105,0.000084
1,2.277191e+07,84102.166772,0.065196,25.656706,0.000268
2,9.175781e+07,31225.232905,0.171639,29.383616,0.000095
3,3.972292e+07,107823.087179,0.047984,62.910787,0.000183
4,1.901094e+07,157633.258648,0.018515,72.599706,0.000309


In [8]:
df.tail()

,k0,k1,damping,velocity,w_max
1995,1.340495e+08,68238.118267,0.081144,71.546504,0.000073
1996,7.674841e+07,118055.199765,0.024334,48.119528,0.000110
1997,1.322220e+07,153852.980762,0.080516,60.188139,0.000402
1998,1.885010e+08,69389.389191,0.194409,10.906861,0.000055
1999,1.941723e+08,66850.480956,0.152075,18.918527,0.000054


In [11]:
# --- 2. Data Pre-processing ---

# Separate the inpute features (x) from the output target (y).
x = df[['k0', 'k1', 'damping', 'velocity']]
y = df['w_max']

In [22]:
# Split data into training + validation (85%) and testing (15%) sets.
x_train_val, x_test, y_train_val, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

In [24]:
# Split the 85% block into training (70%) and validation (15%)
# The new test_size is 15/85 to get 15% of the original total data.
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=(0.15/0.85), random_state=42)

In [ ]:
len(x_train)

1700

In [ ]:
len(x_test)

300

In [63]:
# scale the input features
# We fit the scaler ONLY on the training data to prvent data leakage.
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)


In [26]:
print(F"\nData split complete (70/15/15).")
print(F"Total samples: {len(df)}")
print(F"Training samples: {len(x_train)}")
print(F"Validation samples: {len(x_val)}")
print(F"Testing samples: {len(x_test)}")


Data split complete (70/15/15).
Total samples: 2000
Training samples: 1400
Validation samples: 300
Testing samples: 300


In [ ]:
# --- 3. Build the Neural Network Model ---
print("\nBuilding the Neural Network Model...")



Building the Neural Network Model...


In [ ]:
# Input Layer: The shape must match the number of input features (4).
model = tf.keras.Sequential([tf.keras.layers.Input(shape=(x_train_scaled.shape[1],)),
                             
# First hidden layer with 128 neurons and ReLU activation function.
tf.keras.layers.Dense(128, activation='relu'),

# Second hidden layer with 64 neurons.
tf.keras.layers.Dense(64, activation='relu'),

# Third hidden layer with 32 neurons.
tf.keras.layers.Dense(32, activation='relu'),

# Output Layer: A Single neuron for the regression output (w_max).
tf.keras.layers.Dense(1)
])

In [33]:
# Compile the model.
# We define the Optimizer (Adam), Loss function (Mean Squared Error), and any Metrics to track.
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

In [34]:
# Print a summary of the model architecture.
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,976 (42.88 KB)

 Trainable params: 10,976 (42.88 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# --- 4. Train the Model ---
print("\nTraining the model...")
start_time = time.time()


Training the model...


In [38]:
# Define an EarlyStopping callback to prevent overfitting.
# This will monitor and stop training if the validation loss does not improve for 20 epochs.
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True) # Restores model weights from the best epoch.


In [64]:
# Train the model
# We use 15% of the training data for validation.
# The batch size is set to 32, and we train for a maximum of 500 epochs.
# The `verbose` parameter is set to 1 to display training progress.
history = model.fit(x_train_scaled, y_train, epochs=500, validation_split=0.15, batch_size=32, callbacks=[early_stopping], verbose=[1])

Epoch 1/500


Epoch 2/500
Epoch 3/500
Epoch 4/500
Epoch 5/500
Epoch 6/500
Epoch 7/500
Epoch 8/500
Epoch 9/500
Epoch 10/500
Epoch 11/500
Epoch 12/500
Epoch 13/500
Epoch 14/500
Epoch 15/500
Epoch 16/500
Epoch 17/500
Epoch 18/500
Epoch 19/500
Epoch 20/500
Epoch 21/500


In [60]:
end_time = time.time()
training_time = end_time - start_time
print(F"\nTraining complete.")
print(F"\nTraining time: {training_time:.2f} seconds.")


Training complete.

Training time: 67527.96 seconds.


In [47]:
# --- 5. Evaluate the Model ---
print(F"\nEvaluating the Model on the test set...")


Evaluating the Model on the test set...


In [61]:
# Make predictions on the unseen test data
y_pred = model.predict(x_test_scaled).flatten() # flatten converts a column vector

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [62]:
print(y_test.shape)
print(y_pred.shape)

(300,)
(9600,)


In [52]:
# Calculate performance metrics.
# Ensure y_test and y_pred are both 1D arrays of the same length
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

ValueError: Found input variables with inconsistent numbers of samples: [300, 9600]